In [1]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import sqlalchemy
import utils

In [2]:
# Read the environment variables
%load_ext dotenv
%dotenv ../.env

In [3]:
# Pandas settings
pd.set_option("display.max_colwidth", None)

In [4]:
# Connect to the database
host = os.environ["LOCAL_HOST"]
port = os.environ["LOCAL_PORT"]
dbname = os.environ["POSTGRES_DB"]
dbuser = os.environ["POSTGRES_USER"]
dbpassword = os.environ["POSTGRES_PASSWORD"]

conn_string = f"postgresql://{dbuser}:{dbpassword}@{host}:{port}/{dbname}"
engine = sqlalchemy.create_engine(conn_string)

In [5]:
sql = """
SELECT * 
  FROM information_schema.tables
 WHERE table_schema = 'input'
"""
result = pd.read_sql(sql, engine)
result

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,db2,input,account_info,BASE TABLE,None,None,None,None,None,YES,NO,None
1,db2,input,interactions,BASE TABLE,None,None,None,None,None,YES,NO,None
2,db2,input,product_holdings,BASE TABLE,None,None,None,None,None,YES,NO,None


In [6]:
sql = """
SELECT *
  FROM input.account_info
"""
result = pd.read_sql(sql, engine)
result.head()

,account_id,language,gender,birthday,zip_code,payment_method
0,1000,EN,F,2000-02-25,5005,LSV
1,1001,IT,F,1964-05-17,9102,Kreditkarte
2,1002,DE,F,1964-05-13,3005,LSV
3,1003,None,F,1976-02-01,9102,Kreditkarte
4,1004,DE,M,1968-01-13,4003,Kreditkarte


In [7]:
table_schema = "input"
table_name = "account_info"

output = utils.table_summary(table_schema, table_name, engine)
output

,column_name,data_type,total_count,unique_count,is_unique,min_value,max_value,true_count,false_count,null_count,empty_count,unique_values
0,account_id,bigint,9000,9000,True,1000,9999,None,None,0,0,None
1,language,text,9000,7,False,de,IT,None,None,1536,0,"[de, DE, en, EN, fr, FR, IT, None]"
2,gender,text,9000,3,False,D,M,None,None,0,0,"[D, F, M]"
3,birthday,text,9000,6184,False,1804-04-25,2020-01-30,None,None,0,0,None
4,zip_code,bigint,9000,9,False,1006,9102,None,None,0,0,"[1006, 2006, 3005, 4003, 5005, 6006, 7001, 8006, 9102]"
5,payment_method,text,9000,3,False,Kreditkarte,Rechnung,None,None,0,0,"[Kreditkarte, LSV, Rechnung]"


- account_id is a unique column
- language is case sensitive => convert to lower case
- birthday has values older than 1900
- birthday has values newer than 2008 (i.e. younger than 16 years old)
- No future birthday values
- zip_code range is between 1000 and 9999 (Switzerland)

In [8]:
# Check the language column
sql = """
select distinct language
  from input.account_info
 order by 1
"""
result = pd.read_sql(sql, engine)
result

,language
0,de
1,DE
2,en
3,EN
4,fr
5,FR
6,IT
7,None


In [9]:
# Check the birthday column
sql = """
select min(birthday),
       max(birthday),
       count(case when birthday < '1900-01-01' then 1 end) as count_birthday_before_1900,
       count(case when birthday > '2008-01-01' then 1 end) as count_birthday_after_2008
  from input.account_info
"""
result = pd.read_sql(sql, engine)
result

,min,max,count_birthday_before_1900,count_birthday_after_2008
0,1804-04-25,2020-01-30,69,279


In [10]:
# Check the gender column
sql = """
select gender,
       count(*) as count
  from input.account_info
 group by gender
 order by count desc
"""
result = pd.read_sql(sql, engine)
result.head()

,gender,count
0,M,4325
1,F,4234
2,D,441


In [11]:
# Check the zip_code column
sql = """
select count(*),
       count(distinct zip_code),
       min(zip_code) as min_zip_code,
       max(zip_code) as max_zip_code,
       count(case when zip_code is null then 1 end) as null_zip_code
  from input.account_info
"""
result = pd.read_sql(sql, engine)
result.head()

,count,count,min_zip_code,max_zip_code,null_zip_code
0,9000,9,1006,9102,0


In [12]:
sql = """
SELECT *
  FROM input.product_holdings
"""
result = pd.read_sql(sql, engine)
result.head()

,account_id,subscription_id,product_family,product_name,product_price,valid_from,valid_to
0,8179,46932,Mobile,Plus,23.2,2023-01-04,9999-12-31
1,9195,38294,Fixline,Metered,0.0,2023-04-11,9999-12-31
2,3243,22452,TV,M,7.5,2022-05-12,2023-04-12
3,8680,43936,Mobile,Basic,15.2,2021-04-10,2023-02-28
4,6163,49317,Mobile,Basic,19.0,2021-08-23,2023-04-05


In [13]:
table_schema = "input"
table_name = "product_holdings"

output = utils.table_summary(table_schema, table_name, engine)
output

,column_name,data_type,total_count,unique_count,is_unique,min_value,max_value,true_count,false_count,null_count,empty_count,unique_values
0,account_id,bigint,24593,8940,False,1000,9999,None,None,0,0,None
1,subscription_id,bigint,24593,24456,False,10001,49996,None,None,0,0,None
2,product_family,text,24593,4,False,Fixline,TV,None,None,0,0,"[Fixline, Internet, Mobile, TV]"
3,product_name,text,24593,12,False,100,Unlimited,None,None,0,0,"[100, 1000, 500, Basic, CH Flat, Europe Flat, L, M, Metered, Plus, S, Unlimited]"
4,product_price,double precision,24593,25,False,0.0,49.0,None,None,0,0,None
5,valid_from,text,24593,1000,False,2021-01-01,2023-09-27,None,None,0,0,None
6,valid_to,text,24593,803,False,2021-03-18,9999-12-31,None,None,0,0,None


- product_family has 4 unique values
- product_name has 12 unique values
- prduct_price is between 0.0 and 49.0 (product_price = 0.0 => Fixline, Metered)
- 437 records have valid_to < valid_from
- 137 records have duplicates

In [14]:
# Records with product_price = 0.0
sql = """
SELECT DISTINCT
       product_price,
       array_agg(distinct product_name) as product_names,
       array_agg(distinct product_family) as account_ids,
       count(*)
  FROM input.product_holdings
 WHERE product_price = 0.0
 GROUP BY product_price
"""
result = pd.read_sql(sql, engine)
result

,product_price,product_names,account_ids,count
0,0.0,[Metered],[Fixline],2957


In [15]:
# Average number of products per account
sql = """
SELECT product_family,
       count(*) as count,
       count(*) / sum(count(*)) over() * 100 as percentage
  FROM input.product_holdings
 GROUP BY product_family
 ORDER BY count desc
"""
result = pd.read_sql(sql, engine)
result


,product_family,count,percentage
0,Internet,8220,33.424145
1,TV,6518,26.503477
2,Mobile,5365,21.815151
3,Fixline,4490,18.257228


In [16]:
sql = """
SELECT *
  FROM input.product_holdings
 WHERE valid_to < valid_from
"""
result = pd.read_sql(sql, engine)
result

,account_id,subscription_id,product_family,product_name,product_price,valid_from,valid_to
0,5026,25680,TV,M,7.5,2022-10-27,2022-10-12
1,6843,40300,Mobile,Plus,23.2,2023-05-24,2023-03-31
2,2643,36057,Fixline,Metered,0.0,2023-04-21,2023-03-31
3,8860,14352,Internet,100,19.0,2023-07-11,2022-04-02
4,9241,17404,Internet,1000,39.0,2022-12-02,2022-10-13
...,...,...,...,...,...,...,...
432,2684,20116,TV,L,15.0,2023-04-03,2022-02-08
433,5470,11108,Internet,500,26.1,2023-05-03,2023-04-04
434,6687,45710,Mobile,Plus,23.2,2021-12-17,2021-09-13
435,7250,42083,Mobile,Plus,26.1,2023-06-12,2023-03-31


In [17]:
# Check subscption_id duplicates
sql = """
WITH duplicates as (
  SELECT subscription_id,
        count(*) as count
    FROM input.product_holdings
  GROUP BY subscription_id
  HAVING count(*) > 1
)
SELECT ph.*
  FROM input.product_holdings ph,
       duplicates d
 WHERE ph.subscription_id = d.subscription_id
 ORDER BY ph.subscription_id
"""
result = pd.read_sql(sql, engine)
result

,account_id,subscription_id,product_family,product_name,product_price,valid_from,valid_to
0,9430,10595,Internet,100,19.0,2022-06-14,9999-12-31
1,9430,10595,Internet,100,19.0,2022-06-14,9999-12-31
2,1927,11038,Internet,500,26.1,2021-10-15,9999-12-31
3,1927,11038,Internet,500,26.1,2021-10-15,9999-12-31
4,8822,11168,Internet,500,26.1,2023-06-27,9999-12-31
...,...,...,...,...,...,...,...
269,3705,49729,Mobile,Unlimited,39.2,2021-07-24,2022-08-02
270,3426,49863,Mobile,Basic,19.0,2021-11-18,2023-04-30
271,3426,49863,Mobile,Basic,19.0,2021-11-18,2023-04-30
272,3634,49869,Mobile,Unlimited,49.0,2022-09-21,9999-12-31


In [18]:
# Check that DISTINCT removes the duplicates
sql = """
WITH dedup as (
    SELECT distinct
           ph.*
      FROM input.product_holdings ph
)
SELECT count(*),
       count(distinct subscription_id)
  FROM dedup
"""
result = pd.read_sql(sql, engine)
result.head()

,count,count
0,24456,24456


In [19]:
# All account_id values exists in the account_info table
sql = """
SELECT count(*)
  FROM input.product_holdings ph
 WHERE not exists (
       SELECT 1
         FROM input.account_info ai
        WHERE ai.account_id = ph.account_id
       )
"""
result = pd.read_sql(sql, engine)
result

,count
0,0


In [20]:
# Check (account_id, product_name) has multiple records
# and if so, if they overlap in time
sql = """
SELECT account_id,
       product_name,
       count(*),
       CASE WHEN max(valid_from::date) <= min(valid_to::date) THEN true ELSE false END as overlap
  FROM input.product_holdings
 GROUP BY account_id,
       product_name
HAVING count(*) > 1
"""
result = pd.read_sql(sql, engine)
result

,account_id,product_name,count,overlap
0,8265,Plus,2,False
1,6858,Basic,2,True
2,3490,M,2,True
3,7086,Plus,3,False
4,6812,Plus,2,True
...,...,...,...,...
589,2544,Plus,2,False
590,2376,Unlimited,2,False
591,6771,L,2,True
592,4489,Unlimited,2,True


In [21]:
# Check (account_id, product_name) has unique values
sql = """
SELECT *
  FROM input.product_holdings
 WHERE account_id = 6858
   AND product_name = 'Basic'
 ORDER BY valid_from
"""
result = pd.read_sql(sql, engine)
result

,account_id,subscription_id,product_family,product_name,product_price,valid_from,valid_to
0,6858,45331,Mobile,Basic,17.1,2021-06-03,2023-05-26
1,6858,42411,Mobile,Basic,19.0,2022-08-29,2023-04-26


In [22]:
# Check product_family and product_name
sql = """
SELECT product_family,
       product_name,
       count(*)
  FROM input.product_holdings
 GROUP BY product_family,
       product_name
 ORDER BY product_family,
       product_name
"""
result = pd.read_sql(sql, engine)
result

,product_family,product_name,count
0,Fixline,CH Flat,1318
1,Fixline,Europe Flat,215
2,Fixline,Metered,2957
3,Internet,100,3277
4,Internet,1000,1565
5,Internet,500,3378
6,Mobile,Basic,1603
7,Mobile,Plus,2401
8,Mobile,Unlimited,1361
9,TV,L,2596


In [23]:
sql = """
SELECT *
  FROM input.interactions
"""
result = pd.read_sql(sql, engine)
result.head()

,account_id,date,time_in_queue,handling_time_s,call_reason,customer_satisfaction_after_call
0,1000,2022-11-25,12,42,Billing,NaN
1,1000,2023-01-08,6,135,Technical,5.0
2,1000,2024-06-09,12,135,Technical,2.0
3,1001,2022-09-24,13,82,Billing,4.0
4,1002,2024-01-17,48,26,Billing,NaN


In [24]:
table_schema = "input"
table_name = "interactions"

output = utils.table_summary(table_schema, table_name, engine)
output

,column_name,data_type,total_count,unique_count,is_unique,min_value,max_value,true_count,false_count,null_count,empty_count,unique_values
0,account_id,bigint,14683,7271,False,1000,9999,None,None,0,0,None
1,date,text,14683,1721,False,2021-01-04,2026-05-20,None,None,0,0,None
2,time_in_queue,bigint,14683,57,False,3,90,None,None,0,0,None
3,handling_time_s,bigint,14683,107,False,11,186,None,None,0,0,None
4,call_reason,text,14683,4,False,Billing,Technical,None,None,303,0,"[Billing, Churn, Product, Technical, None]"
5,customer_satisfaction_after_call,double precision,14683,5,False,1.0,5.0,None,None,5860,0,"[1.0, 2.0, 3.0, 4.0, 5.0, None]"


- The value range of the date column is between 2021-01-04 and 2026-05-26 (Future interactions exist)
- The value range of the time_in_queue column is between 3 and 90
- The value range of the handling_time_s column between 11 and 186
- The call_reason values are Billing, Churn, Product, Technical and NULL
- The customer_satisfaction_after_call values are 1.0, 2.0, 3.0, 4.0, 5.0 and NULL

In [25]:
# date newer than today
sql = """
SELECT *
  FROM input.interactions
 WHERE date::date > current_date
 ORDER BY date
"""
result = pd.read_sql(sql, engine)
result

,account_id,date,time_in_queue,handling_time_s,call_reason,customer_satisfaction_after_call
0,5774,2024-02-27,40,86,Billing,2.0
1,8547,2024-02-27,39,102,Billing,5.0
2,7685,2024-02-27,45,162,Product,NaN
3,8167,2024-02-27,51,70,Technical,NaN
4,2009,2024-02-27,57,78,Product,2.0
...,...,...,...,...,...,...
2448,3718,2026-04-07,26,49,Product,NaN
2449,5089,2026-04-19,33,96,Churn,NaN
2450,8118,2026-04-28,11,62,Churn,5.0
2451,2534,2026-05-14,26,126,Technical,NaN


In [26]:
# All account_id values exists in the account_info table
sql = """
SELECT count(*)
  FROM input.interactions i
 WHERE not exists (
       SELECT 1
         FROM input.account_info ai
        WHERE ai.account_id = i.account_id
       )
"""
result = pd.read_sql(sql, engine)
result

,count
0,0
